In [2]:
import numpy as np
import tensorflow as tf
from digits_utils import *

In [3]:
from keras.layers import Input, Add, Dense, Conv2D, Activation, BatchNormalization, ZeroPadding2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from keras.models import Model

In [4]:
train_it=load_images()

Found 7557 images belonging to 16 classes.


In [5]:
train_it.next()[0][0].shape

(64, 64, 3)

In [6]:
train_it.next()[1].shape

(32, 16)

In [7]:
### classes_size=16

In [8]:
def identity_block(a_prev, filters, stride, kernel_size):
    
    F1, F2 =filters              ###  F2 must be equal to c    if a_prev has shape(n,n,c)
    
    s=stride
    
    k= kernel_size
    
    a_input=a_prev
    
    X=Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid', kernel_initializer=glorot_uniform(seed=0))(a_prev)
    
    X=BatchNormalization(axis=3)(X)   ### batchnormalization through channels.....................
    
    X=Activation('relu')(X)
    
    X=Conv2D(filters=F2, kernel_size=(k,k), strides=(1,1), padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
    
    X=BatchNormalization(axis=3)(X)
    
    X=Add()([a_input, X])
    
    X=Activation('relu')(X)
    
    return X
    

In [9]:
def convolutional_block(a_prev, stride, filters, kernel_size):
    
    
    F1, F2 =filters
    
    k=kernel_size
    s=stride
    
    a_input=a_prev
    
   
    X=Conv2D(filters=F1, kernel_size=(1,1), strides=(s,s), padding='valid', kernel_initializer=glorot_uniform(seed=0))(a_prev)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X=Conv2D(filters=F2, kernel_size=(k,k), strides=(1,1), padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3)(X)
    
    
    a_input= Conv2D(filters=F2, kernel_size=(1,1), strides=(s,s), padding='valid', kernel_initializer=glorot_uniform(seed=0))(a_input)
    a_input=BatchNormalization(axis=3)(a_input)
    
    X=Add()([a_input, X])
    X=Activation('relu')(X)
    
    return X
    

In [10]:
def model(input_shape, filters, stride, kernel_size, classes_size):
    
    
    X_input=Input(shape=input_shape)
    
    X=ZeroPadding2D(padding=(3,3))(X_input)
    
    X=Conv2D(filters=64 , kernel_size=(5,5), padding='valid', strides=(2,2), kernel_initializer= glorot_uniform (seed=0))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X=MaxPooling2D(pool_size=(3,3), strides=(2,2))(X)
    
    X= convolutional_block(X, stride, filters, kernel_size)
    X= identity_block(X, filters, stride, kernel_size)
    X= identity_block(X, filters, stride, kernel_size)
    
    X= convolutional_block(X, stride, filters, kernel_size)
    X= identity_block(X, filters, stride, kernel_size)
    X= identity_block(X, filters, stride, kernel_size)
    X= identity_block(X, filters, stride, kernel_size)
    
    
    X=Flatten()(X)
    
    #X=Dense(units=128, activation='relu')(X)
    
    X=Dense(units=classes_size, activation='softmax')(X)
    
    model=Model(inputs=X_input, outputs=X)
    
    return model
    

In [11]:
model=model(input_shape=(64,64,3), filters=[3,5], stride=2, kernel_size=3, classes_size=16)

In [12]:
opt=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit_generator(train_it, steps_per_epoch=32, epochs=20)

Epoch 1/20
32/32 [==============================] - 33s - loss: 3.2041 - acc: 0.1660    
Epoch 2/20
32/32 [==============================] - 29s - loss: 2.3326 - acc: 0.2422    
Epoch 3/20
32/32 [==============================] - 26s - loss: 2.1790 - acc: 0.3018    
Epoch 4/20
32/32 [==============================] - 25s - loss: 1.9294 - acc: 0.3916    
Epoch 5/20
32/32 [==============================] - 23s - loss: 1.7994 - acc: 0.4326    
Epoch 6/20
32/32 [==============================] - 44s - loss: 1.6992 - acc: 0.4512    
Epoch 7/20
32/32 [==============================] - 22s - loss: 1.5408 - acc: 0.5127    
Epoch 8/20
32/32 [==============================] - 15s - loss: 1.4153 - acc: 0.5351    
Epoch 9/20
32/32 [==============================] - 13s - loss: 1.2985 - acc: 0.5732    
Epoch 10/20
32/32 [==============================] - 12s - loss: 1.3712 - acc: 0.5566    
Epoch 11/20
32/32 [==============================] - 11s - loss: 1.2494 - acc: 0.5908    
Epoch 12/20
32/32 [

In [14]:
eval_it=load_eval_data()

Found 1010 images belonging to 16 classes.


In [25]:
loss, accuracy= model.evaluate_generator(eval_it, steps=32)

In [26]:
print("loss :%f"% loss)
print("accuracy :%f"% accuracy)

loss :1.007124
accuracy :0.653465
